**Добрый день, коллеги!**

Первичная задача - детектить похожие ответы на один и тот же вопрос в автоматическом режиме

Бейзлайн - переводим текст в эмбеддинги с помощью русского берта и [huggingface](https://huggingface.co/). Считаем обратное косинусное расстояние между ответами банковских сотрудников (cosine similarity). 

Чем ближе полученное значение к единице - тем больше ответы похожи друг на друга и близки по смыслу. 

На первом этапе будем отбирать первые 4 ответа консультантов или менеджеров после вопроса клиента. 

Первичные проблемы - положительные, но различные ответы сотрудников могут быть близки друг другу по косинусным расстояниям. В текущем примере я установил очень большой порог схожести (threshold = 0.980). После теста на реальных данных можно дотюнить параметр.  

Для корректной работы, отбирайте и предобрабатывайте чаты моими функциями из прошлого ноутбука. Главное - чтобы фразы были выстроены в хронологическом порядке. В отличие от первого ноутбука, здесь маркировка сообщений сделана проще.

---

С Эмбеддингами можно работать локально или каждый раз автоматически подтягивать модель с интернета.

**Для работы локально.** [Cсылка](https://huggingface.co/DeepPavlov/rubert-base-cased/tree/main) для скачивания русскоязычного берта. Забираем все файлы, кроме flax_model.msgpack и сохраняем в одну папку. В ячейке ниже меняем PATH на путь к папке со скачанными файлыми.  

**Для подтягивания модели с интернета**. В ячейке 2 заменяем PATH на 'rubert-base-cased'. Скачиваение начнется в момент выполнения кода в ячейке. При реране ноутбука на новом кернеле требуется скачивать файлы заново.  

**Важно**  
Для работы на локальной рабочей машине требуется torch и huggingface (transformers). Torch есть на внутреннем ресурсе, но у меня криво устанавливался - требовалась дополнительная установка visual C++. На счет huggingface (transformers) - не знаю.

---

In [1]:
import numpy as np
import pandas as pd
import re
import torch
from tqdm import notebook
from itertools import combinations
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist, cosine

PATH = 'E:\\_work\\BERTS\\rubert_base_cased'

In [2]:
tokenizer = AutoTokenizer.from_pretrained(PATH)
model = AutoModel.from_pretrained(PATH)

In [3]:
texts = [
    'Вы втираете мне какую то дичь!!', 
    'Что это за дерьмо?',
    'Добрый день. Когда могу подписать договор?',
    'Здравствуйте. Завтра в 18 часов по адресу Лесной проспект, дом 68. При себе необходимо иметь паспот и договор.',
    'Я проверил, вам предодобрена ипотека в размере 100500 млн рублей.',
    'Спасибо.',
    'Вы еще тут?',
    'Спасибо за обращение. Мы ценим ваше отношение к нашему сервису.',
    'Добрый день. Ждем вас на следующей неделе в 16 часов по адресу Лесной проспект, дом 68. При себе необходимо иметь паспот и договор.',
    'Добрый день. Приезжайте через 4 дня в 19 часов по адресу Лесной проспект, дом 68. Не забудьте паспорт и договор.',
    'Вы втираете мне какую то ересь']

users = [
    'USER',
    'CONSULTANT',
    'USER',
    'COSNULTANT',
    'COSNULTANT',
    'USER',
    'USER',
    'BOT',
    'CONSULTANT',
    'CONSULTANT',
    'USER'
]

df = pd.DataFrame(data={'text':texts, 'users':users})
df

,text,users
0,Вы втираете мне какую то дичь!!,USER
1,Что это за дерьмо?,CONSULTANT
2,Добрый день. Когда могу подписать договор?,USER
3,Здравствуйте. Завтра в 18 часов по адресу Лесн...,COSNULTANT
4,"Я проверил, вам предодобрена ипотека в размере...",COSNULTANT
5,Спасибо.,USER
6,Вы еще тут?,USER
7,Спасибо за обращение. Мы ценим ваше отношение ...,BOT
8,Добрый день. Ждем вас на следующей неделе в 16...,CONSULTANT
9,Добрый день. Приезжайте через 4 дня в 19 часов...,CONSULTANT


In [4]:
def add_mark(row):
    
    '''
    Разбивает 
    '''
    
    doc = row['text']
    if row['users'] == 'USER' and len(re.findall(r'\?', doc)) > 0:
        return 'user_question'
    elif row['users'] == 'USER' and len(re.findall(r'\?', doc)) == 0:
        return 'user_answer'
    elif row['users'] != 'USER' and len(re.findall(r'\?', doc)) == 0:
        return 'bank_answer'
    else:
        return 'bank_question'

In [5]:
df['marked'] = df.apply(add_mark, axis=1)

In [6]:
df

,text,users,marked
0,Вы втираете мне какую то дичь!!,USER,user_answer
1,Что это за дерьмо?,CONSULTANT,bank_question
2,Добрый день. Когда могу подписать договор?,USER,user_question
3,Здравствуйте. Завтра в 18 часов по адресу Лесн...,COSNULTANT,bank_answer
4,"Я проверил, вам предодобрена ипотека в размере...",COSNULTANT,bank_answer
5,Спасибо.,USER,user_answer
6,Вы еще тут?,USER,user_question
7,Спасибо за обращение. Мы ценим ваше отношение ...,BOT,bank_answer
8,Добрый день. Ждем вас на следующей неделе в 16...,CONSULTANT,bank_answer
9,Добрый день. Приезжайте через 4 дня в 19 часов...,CONSULTANT,bank_answer


In [7]:
df.query('marked == "bank_answer"').index

Int64Index([3, 4, 7, 8, 9], dtype='int64')

In [8]:
df[3:].query('marked == "bank_answer" and users != "BOT"').index.values.tolist()[:4]

[3, 4, 8, 9]

In [9]:
def answer_index(df):
    '''
    Сохраняет следующие 4 ответа банка после вопроса клиента от реальных сотрудников и НЕ бота
    '''
    prob_answers = {}
    answers = []

    for i in range(len(df)):
        if df.loc[i, 'marked'] == 'user_question':
            answers = df[i:].query('marked == "bank_answer" and users != "BOT"').index.tolist()[:4]
            prob_answers[i] = answers
        
    return prob_answers

In [10]:
question_answers = answer_index(df)
question_answers

{2: [3, 4, 8, 9], 6: [8, 9]}

In [11]:
corpus = df['text'].str.lower().values

In [12]:
tokenized = []
for i in corpus:
    tokenized.append(tokenizer.encode(i, add_special_tokens=True))

In [13]:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized])
#padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized].values)

attention_mask = np.where(padded != 0, 1, 0)

In [14]:
batch_size = 1
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size * i : batch_size * (i + 1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size * i : batch_size * (i + 1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [24]:
 cosine_similarity(embeddings[3], embeddings[4])

array([[0.9594257]], dtype=float32)

In [15]:
 cosine_similarity(embeddings[7], embeddings[6])

array([[0.97552633]], dtype=float32)

In [43]:
threshold = 0.980
list_answers = []
sublist = []

for i in question_answers.keys():
    combi = list(combinations(question_answers[i], 2))
    
    for j in combi:
        similarity = cosine_similarity(embeddings[j[0]], embeddings[j[1]])
        if similarity >= threshold:
            sublist.append(i)
            sublist.append(j[0])
            sublist.append(j[1])
            sublist.append(similarity[0][0])
            list_answers.append(sublist)
            sublist = []

In [44]:
list_answers

[[2, 3, 8, 0.9825882],
 [2, 3, 9, 0.9880197],
 [2, 8, 9, 0.9898157],
 [6, 8, 9, 0.9898157]]

In [47]:
list_answers[0][0]

2

In [52]:
for i in range(len(list_answers)):
    print('Вопрос: ' + corpus[list_answers[i][0]])
    print('Ответ: '+ corpus[list_answers[i][1]])
    print('Ответ: '+ corpus[list_answers[i][2]])
    print('Косинусное = ', list_answers[i][3])
    print('-----')

Вопрос: добрый день. когда могу подписать договор?
Ответ: здравствуйте. завтра в 18 часов по адресу лесной проспект, дом 68. при себе необходимо иметь паспот и договор.
Ответ: добрый день. ждем вас на следующей неделе в 16 часов по адресу лесной проспект, дом 68. при себе необходимо иметь паспот и договор.
Косинусное =  0.9825882
-----
Вопрос: добрый день. когда могу подписать договор?
Ответ: здравствуйте. завтра в 18 часов по адресу лесной проспект, дом 68. при себе необходимо иметь паспот и договор.
Ответ: добрый день. приезжайте через 4 дня в 19 часов по адресу лесной проспект, дом 68. не забудьте паспорт и договор.
Косинусное =  0.9880197
-----
Вопрос: добрый день. когда могу подписать договор?
Ответ: добрый день. ждем вас на следующей неделе в 16 часов по адресу лесной проспект, дом 68. при себе необходимо иметь паспот и договор.
Ответ: добрый день. приезжайте через 4 дня в 19 часов по адресу лесной проспект, дом 68. не забудьте паспорт и договор.
Косинусное =  0.9898157
-----
Воп

Далее уже смотрим глазками и отдаем аналитикам.